In [1]:
from bioio import BioImage
import numpy as np
import os
from glob import glob
import skimage as sk
import napari
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt
from natsort import natsorted

In [2]:
test_files = sorted(glob('../../**/*.czi'))
czi_files = list(map(BioImage,test_files))


In [3]:
len(czi_files)

8

Split images for CellPose Training

In [5]:
c=1
for file in czi_files:
    img = file.data
    img = np.squeeze(img)
    ch1 = img[0,...]
    ch1_inverted = sk.util.invert(ch1)
    ch2 = img[1,...]
    ch2_inverted = sk.util.invert(ch2)
    sk.io.imsave(os.path.join('../../CellPose_Training_3D/','inverted_ch1_img_0'+str(c)+'.tiff'),ch1_inverted)
    sk.io.imsave(os.path.join('../../CellPose_Training_3D/','inverted_ch2_img_0'+str(c)+'.tiff'),ch2_inverted)
    c+=1

c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ../../CellPose_Training_3D/inverted_ch1_img_01.tiff is a low contrast image
  return func(*args, **kwargs)
c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ../../CellPose_Training_3D/inverted_ch2_img_01.tiff is a low contrast image
  return func(*args, **kwargs)
c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ../../CellPose_Training_3D/inverted_ch1_img_02.tiff is a low contrast image
  return func(*args, **kwargs)
c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ../../CellPose_Training_3D/inverted_ch2_img_02.tiff is a low contrast image
  return func(*args, **kwargs)
c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ../../CellPose_Training_3D/inverted_c

In [11]:
img.shape

(2, 71, 1000, 1000)

In [22]:
ch1_img = np.squeeze(ch1_img)

In [7]:
viewer = napari.view_image(img, scale=[czi_file.physical_pixel_sizes.Z,czi_file.physical_pixel_sizes.X,czi_file.physical_pixel_sizes.Y])

C:\Users\Kristin.Gallik\AppData\Local\Temp\ipykernel_5948\1044138916.py:1: FutureWarning: `napari.view_image` is deprecated and will be removed in napari 0.7.0.
Use `viewer = napari.Viewer(); viewer.add_image(...)` instead.
  viewer = napari.view_image(img, scale=[czi_file.physical_pixel_sizes.Z,czi_file.physical_pixel_sizes.X,czi_file.physical_pixel_sizes.Y])


In [3]:
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

model = models.CellposeModel(gpu=True)

In [ ]:
first_channel = 0
second_channel = 1
selected_channels = []
for i, c in enumerate([first_channel, second_channel]):
  if c == 'None':
    continue
  if int(c) > img.shape[0]:
    assert False, 'invalid channel index, must have index greater or equal to the number of channels'
  if c != 'None':
    selected_channels.append(int(c))



img_selected_channels = np.zeros_like(img)
img_selected_channels[:, :, :len(selected_channels)] = img[:, :, selected_channels]

io.logger_setup()
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0

masks, flows, styles = model.eval(img_selected_channels, channel_axis=0, z_axis=1, batch_size=32, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold,
                                  normalize={"tile_norm_blocksize": tile_norm_blocksize},do_3D=True)

fig = plt.figure(figsize=(12,5))
plot.show_segmentation(fig, img_selected_channels, masks, flows[0])
plt.tight_layout()
plt.show()

2025-08-16 11:41:40,924 [INFO] WRITING LOG OUTPUT TO C:\Users\Kristin.Gallik\.cellpose\run.log
2025-08-16 11:41:40,926 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.8.0+cu126
2025-08-16 11:41:42,993 [INFO] running YX: 71 planes of size (1000, 1000)
2025-08-16 11:52:06,091 [INFO] 4%|4         | 3/71 [10:23<3:55:23, 207.70s/it]


In [6]:
mask_files = sorted(glob("../../CellPose_Training_3D/*_masks.tif"))
masks = list(map(sk.io.imread,mask_files))

In [7]:
cleared_boarders = [sk.segmentation.clear_border(mask) for mask in masks]

In [9]:
removed_small_objects = [sk.morphology.remove_small_objects(img, min_size=50) for img in cleared_boarders]

In [10]:
viewer = napari.view_image(removed_small_objects[1])

In [11]:
c=1
for img in removed_small_objects:
    sk.io.imsave(os.path.join('../../CellPose_Training_3D/','ch1_img_0'+str(c)+'_filtered_masks.tif'),img,check_contrast=False)
    c+=1